In [7]:
# Delete this cell to re-enable tracebacks
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Creating STIX Content

### Creating STIX Domain Objects

To create a STIX object, provide keyword arguments to the type's constructor:

In [1]:
from stix2 import Indicator

indicator = Indicator(name="File hash for malware variant",
                      labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")
print(indicator)

{
    "type": "indicator",
    "id": "indicator--38feb147-4277-45d7-b16c-5f60e24b88e1",
    "created": "2017-09-14T18:17:15.709Z",
    "modified": "2017-09-14T18:17:15.709Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "File hash for malware variant",
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-14T18:17:15.709845Z"
}


Certain required attributes of all objects will be set automatically if not provided as keyword arguments:

-  If not provided, ``type`` will be set automatically to the correct type. You can also provide the type explicitly, but this is not necessary:

In [8]:
indicator2 = Indicator(type='indicator',
                       labels=["malicious-activity"],
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

Passing a value for ``type`` that does not match the class being constructed will cause an error:

In [9]:
indicator3 = Indicator(type='xxx',
                       labels=["malicious-activity"],
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

InvalidValueError: Invalid value for Indicator 'type': must equal 'indicator'.

-  If not provided, ``id`` will be generated randomly. If you provide an
   ``id`` argument, it must begin with the correct prefix:

In [11]:
indicator4 = Indicator(id="campaign--63ce9068-b5ab-47fa-a2cf-a602ea01f21a",
                       labels=["malicious-activity"],
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

InvalidValueError: Invalid value for Indicator 'id': must start with 'indicator--'.

For indicators, ``labels`` and ``pattern`` are required and cannot be set automatically. Trying to create an indicator that is missing one of these properties will result in an error:

In [12]:
indicator = Indicator()

MissingPropertiesError: No values for required properties for Indicator: (labels, pattern).

However, the required ``valid_from`` attribute on Indicators will be set to the current time if not provided as a keyword argument.

Once created, the object acts like a frozen dictionary. Properties can be accessed using the standard Python dictionary syntax:

In [15]:
indicator['name']

u'File hash for malware variant'

Or access properties using the standard Python attribute syntax:

In [16]:
indicator.name

u'File hash for malware variant'

Attempting to modify any attributes will raise an error:

In [17]:
indicator['name'] = "This is a revised name"

TypeError: 'Indicator' object does not support item assignment

In [18]:
indicator.name = "This is a revised name"

ImmutableError: Cannot modify 'name' property in 'Indicator' after creation.

To update the properties of an object, see the **Versioning** section.

Creating a Malware object follows the same pattern:

In [21]:
from stix2 import Malware

malware = Malware(name="Poison Ivy",
                  labels=['remote-access-trojan'])
print(malware)

{
    "type": "malware",
    "id": "malware--b2eca08d-705a-4662-9b58-9ffe6a98cecd",
    "created": "2017-09-14T18:38:09.395Z",
    "modified": "2017-09-14T18:38:09.395Z",
    "name": "Poison Ivy",
    "labels": [
        "remote-access-trojan"
    ]
}


As with indicators, the ``type``, ``id``, ``created``, and ``modified`` properties will be set automatically if not provided. For Malware objects, the ``labels`` and ``name`` properties must be provided.

### Creating Relationships

STIX 2 Relationships are separate objects, not properties of the object on either side of the relationship. They are constructed similarly to other STIX objects. The ``type``, ``id``, ``created``, and ``modified`` properties are added automatically if not provided. Callers must provide the ``relationship_type``, ``source_ref``, and ``target_ref`` properties.

In [23]:
from stix2 import Relationship

relationship = Relationship(relationship_type='indicates',
                            source_ref=indicator.id,
                            target_ref=malware.id)
print(relationship)

{
    "type": "relationship",
    "id": "relationship--c42147ec-827a-492d-98d1-33eaaf4678e7",
    "created": "2017-09-14T18:39:40.965Z",
    "modified": "2017-09-14T18:39:40.965Z",
    "relationship_type": "indicates",
    "source_ref": "indicator--38feb147-4277-45d7-b16c-5f60e24b88e1",
    "target_ref": "malware--b2eca08d-705a-4662-9b58-9ffe6a98cecd"
}


The ``source_ref`` and ``target_ref`` properties can be either the ID's of other STIX objects, or the STIX objects themselves. For readability, Relationship objects can also be constructed with the ``source_ref``, ``relationship_type``, and ``target_ref`` as positional (non-keyword) arguments:

In [24]:
relationship2 = Relationship(indicator, 'indicates', malware)
print(relationship2)

{
    "type": "relationship",
    "id": "relationship--0097e73a-6340-4394-a499-74e34dcd52fa",
    "created": "2017-09-14T18:40:05.097Z",
    "modified": "2017-09-14T18:40:05.097Z",
    "relationship_type": "indicates",
    "source_ref": "indicator--38feb147-4277-45d7-b16c-5f60e24b88e1",
    "target_ref": "malware--b2eca08d-705a-4662-9b58-9ffe6a98cecd"
}


### Creating Bundles

STIX Bundles can be created by passing objects as arguments to the Bundle constructor. All required properties (``type``, ``id``, and ``spec_version``) will be set automatically if not provided, or can be provided as keyword arguments:

In [26]:
from stix2 import Bundle

bundle = Bundle(indicator, malware, relationship)
print(bundle)

{
    "type": "bundle",
    "id": "bundle--54d649a4-5ab4-4ab6-b4c4-b52c09fbee9c",
    "spec_version": "2.0",
    "objects": [
        {
            "type": "indicator",
            "id": "indicator--38feb147-4277-45d7-b16c-5f60e24b88e1",
            "created": "2017-09-14T18:17:15.709Z",
            "modified": "2017-09-14T18:17:15.709Z",
            "labels": [
                "malicious-activity"
            ],
            "name": "File hash for malware variant",
            "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
            "valid_from": "2017-09-14T18:17:15.709845Z"
        },
        {
            "type": "malware",
            "id": "malware--b2eca08d-705a-4662-9b58-9ffe6a98cecd",
            "created": "2017-09-14T18:38:09.395Z",
            "modified": "2017-09-14T18:38:09.395Z",
            "name": "Poison Ivy",
            "labels": [
                "remote-access-trojan"
            ]
        },
        {
            "type": "relationship",
   